<a href="https://colab.research.google.com/github/sipocz/makerere/blob/main/zindi_1_20220501142000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install Augmentor

In [ ]:
url="https://api.zindi.africa/v1/competitions/makerere-fall-armyworm-crop-challenge/files/Images.zip"
url_train="https://api.zindi.africa/v1/competitions/makerere-fall-armyworm-crop-challenge/files/Train.csv"
url_test="https://api.zindi.africa/v1/competitions/makerere-fall-armyworm-crop-challenge/files/Test.csv"
url_sample_s="https://api.zindi.africa/v1/competitions/makerere-fall-armyworm-crop-challenge/files/SampleSubmission.csv"
auth={'auth_token': "KXxbUBzf4GQ6pqkNGC8zJt8B"}


In [ ]:
# Function to download data
def zindi_data_downloader(data_url, auth, file_name):
    import requests
    from tqdm.auto import tqdm
    # Get the competition data
    x = requests.post(data_url, data = auth,stream=True)
    target_path = 'data.zip'

    handle = open(file_name, "wb")
    for chunk in x.iter_content(chunk_size=512):
        if chunk:  # filter out keep-alive new chunks
            handle.write(chunk)
    handle.close()

In [ ]:
zindi_data_downloader(url,auth,"Images.zip")

In [ ]:
!mkdir pic

In [ ]:
!unzip -q  /content/Images.zip -d /content/pic

In [ ]:
!rm *.zip

In [ ]:
zindi_data_downloader(url_train,auth,"Train.csv")
zindi_data_downloader(url_test,auth,"Test.csv")
zindi_data_downloader(url_sample_s,auth,"SampleSubmission.csv")



In [34]:
import pandas as pd

In [35]:
df=pd.read_csv("Train.csv")
df["fullpath"]="/content/pic/"+df["Image_id"]
o=["zero" if i==0 else "one" for i in df["Label"].values ]
df["cat"]=o

In [36]:
df.head()

,Image_id,Label,fullpath,cat
0,id_02amazy34fgh2.jpg,1,/content/pic/id_02amazy34fgh2.jpg,one
1,id_02mh3w48pmyc9.jpg,0,/content/pic/id_02mh3w48pmyc9.jpg,zero
2,id_02rpb463h9d3w.jpg,0,/content/pic/id_02rpb463h9d3w.jpg,zero
3,id_02wc3jeeao8ol.jpg,1,/content/pic/id_02wc3jeeao8ol.jpg,one
4,id_03t2hapb8wz8p.jpg,1,/content/pic/id_03t2hapb8wz8p.jpg,one


In [37]:
ids=df["Image_id"].to_list()
labels=df["Label"].to_list()

In [38]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D,Dense,Dropout
from tensorflow.keras.models  import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [39]:
from numpy.ma.core import resize
x_px=300
y_px=300
bpc=3
data_path="/content/pic/"
batch_size=10
train=ImageDataGenerator(horizontal_flip=True, vertical_flip=True,validation_split=0.1,rescale=1./255,
                         shear_range = 0.05,zoom_range = 0.05,
                         width_shift_range = 0.0,
                         height_shift_range = 0.0,)

test=ImageDataGenerator(rescale=1./255,validation_split=0.1)




print(f"X: {x_px}, Y: {y_px}")
train_generator=train.flow_from_dataframe(df,x_col="fullpath",y_col="cat",target_size=(x_px,y_px),batch_size=batch_size,class_mode='binary',subset='training',color_mode="rgb")
test_generator = test.flow_from_dataframe(df,x_col="fullpath",y_col="cat",target_size=(x_px,y_px),batch_size=batch_size,class_mode='binary',subset='validation',color_mode="rgb")


X: 300, Y: 300
Found 1458 validated image filenames belonging to 2 classes.
Found 161 validated image filenames belonging to 2 classes.


In [40]:
# Build Model
# Importáld a megfelelő rétegeket
from tensorflow.keras.layers import Input,Dense,Embedding,LSTM,TimeDistributed, Flatten, Bidirectional, Conv2D, Dropout,Convolution2D, Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D, Rescaling,RandomFlip,RandomRotation, Reshape,BatchNormalization,MaxPooling3D

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adadelta,Adam,SGD,Nadam,Ftrl,SGD,Adamax
from tensorflow.keras.losses import sparse_categorical_crossentropy,categorical_crossentropy, mean_squared_error
from tensorflow.keras import backend as K
from keras.regularizers import l1
from keras.initializers import RandomNormal,Zeros,GlorotUniform

K.clear_session()



inputs= Input(shape=(x_px,y_px,bpc))

e = Conv2D(32, (2, 2), activation='relu',)(inputs)
e = MaxPooling2D((2, 2))(e)

e = Conv2D(32, (2, 2), activation='relu')(e)
e = MaxPooling2D((2, 2))(e)

e = Conv2D(32, (2, 2), activation='relu')(e)
e = MaxPooling2D((2, 2))(e)

e = Conv2D(32, (2, 2), activation='relu')(e)
e = MaxPooling2D((2, 2))(e)


l = Flatten()(e)
        #l = Dense(1920, activation='sigmoid')(l)
         # overfitting compensation ??
        #l=Dropout(0.7)(l)
        #l = Dense(74, activation='sigmoid')(l)
    #l=Dropout(0.6)(l)
    #l = Dense(56, activation='sigmoid')(l)
l=Dropout(0.35)(l)  # 0,95 95%

l=Dense(50,activation="relu")(l)

l=Dropout(0.35)(l)  # 0,95 95%


decoded = Dense(2, activation='softmax')(l)


convmodel = Model(inputs, decoded)


In [41]:

cce = tf.keras.losses.SparseCategoricalCrossentropy()
acc = tf.keras.metrics.SparseCategoricalAccuracy()
optimizer=Adam(learning_rate=0.0005)
convmodel.compile(loss=cce, optimizer=optimizer, metrics=[acc])
convmodel.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 300, 300, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 299, 299, 32)      416       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 148, 148, 32)      4128      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 73, 73, 32)        4128  

In [ ]:
history = convmodel.fit(train_generator, epochs=100,  validation_data=test_generator,batch_size=1)

Epoch 1/100
146/146 [==============================] - 55s 366ms/step - loss: 0.6759 - sparse_categorical_accuracy: 0.5816 - val_loss: 0.6036 - val_sparse_categorical_accuracy: 0.7764
Epoch 2/100
146/146 [==============================] - 54s 372ms/step - loss: 0.5221 - sparse_categorical_accuracy: 0.7538 - val_loss: 0.4055 - val_sparse_categorical_accuracy: 0.8261
Epoch 3/100
146/146 [==============================] - 54s 367ms/step - loss: 0.4292 - sparse_categorical_accuracy: 0.8134 - val_loss: 0.3821 - val_sparse_categorical_accuracy: 0.8199
Epoch 4/100
146/146 [==============================] - 54s 367ms/step - loss: 0.4015 - sparse_categorical_accuracy: 0.8292 - val_loss: 0.3418 - val_sparse_categorical_accuracy: 0.8571
Epoch 5/100
146/146 [==============================] - 53s 364ms/step - loss: 0.3700 - sparse_categorical_accuracy: 0.8409 - val_loss: 0.3811 - val_sparse_categorical_accuracy: 0.8261
Epoch 6/100
146/146 [==============================] - 53s 363ms/step - loss: 0.

In [ ]:
def plot_history(history):
    import matplotlib.pyplot as plt
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
plot_history(history)